<a href="https://colab.research.google.com/github/rename-z/Deep-Learning/blob/master/LSTMs/Multiclass%20Text%20Classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import torch

In [2]:
#input 
x = torch.tensor([[1,2, 12,34, 56,78, 90,80],
                  [12,45, 99,67, 6,23, 77,82],
                  [3,24, 6,99, 12,56, 21,22]
                  ])

In [3]:
x

tensor([[ 1,  2, 12, 34, 56, 78, 90, 80],
        [12, 45, 99, 67,  6, 23, 77, 82],
        [ 3, 24,  6, 99, 12, 56, 21, 22]])

### using two different models

In [4]:
import torch.nn as nn

In [5]:
# an Embedding module containing 100 tensors of size 7
model1 = nn.Embedding(100, 7, padding_idx=0)

model2 = nn.LSTM(input_size=7, hidden_size=3, num_layers=1, batch_first=True)

In [6]:
out1 = model1(x)
out2 = model2(out1)

In [7]:
print(out1.shape)
print(out1)

torch.Size([3, 8, 7])
tensor([[[ 6.3114e-01,  1.5083e+00,  2.9205e-01, -1.6661e+00, -8.4888e-01,
          -2.2438e-01,  4.7769e-01],
         [ 2.1364e+00, -6.8609e-01, -8.1242e-01,  2.0382e-01, -3.9472e-01,
           8.9181e-01,  3.5066e-01],
         [-7.8021e-01,  4.3960e-01, -2.5498e-01,  1.3585e+00, -1.9595e+00,
           9.0058e-01,  1.3656e+00],
         [-1.0021e+00, -2.5192e+00, -1.5666e+00,  7.9189e-01,  9.0697e-02,
           3.6672e-02,  7.2193e-01],
         [ 6.7286e-01, -1.6653e+00,  8.0300e-01,  6.2598e-01, -6.7528e-04,
           1.3160e+00,  1.0305e-01],
         [-1.0674e+00, -1.6457e+00, -1.7386e+00, -4.5385e-01,  1.9436e+00,
          -1.5112e+00,  2.5445e-01],
         [ 2.3096e-01, -1.5563e+00, -2.2596e+00,  5.0120e-01, -3.6723e-01,
          -6.8101e-01, -1.7118e+00],
         [ 1.5691e+00,  2.1182e+00,  7.3919e-01,  7.6921e-01, -1.8725e+00,
          -1.5261e-01,  1.4287e+00]],

        [[-7.8021e-01,  4.3960e-01, -2.5498e-01,  1.3585e+00, -1.9595e+00,
     

In [8]:
out, (ht, ct) = model2(out1)
print(ht)

tensor([[[ 0.1457, -0.0435,  0.1359],
         [ 0.1180,  0.4630,  0.2744],
         [-0.0027, -0.3567,  0.3229]]], grad_fn=<StackBackward>)


### using nn.squential

In [9]:
model3 = nn.Sequential(nn.Embedding(100, 7, padding_idx=0),
                       nn.LSTM(input_size=7, hidden_size=3, num_layers=1, batch_first=True))

In [10]:
out, (ht, ct) = model3(x)
print(out)

tensor([[[-0.0342,  0.2533, -0.0840],
         [-0.1244, -0.1821, -0.0704],
         [ 0.0837,  0.2099, -0.1763],
         [ 0.1518,  0.2766, -0.3584],
         [ 0.0231,  0.0897, -0.3447],
         [-0.1021,  0.0009, -0.0205],
         [-0.1646, -0.0772, -0.2228],
         [-0.0377,  0.1878,  0.2741]],

        [[ 0.0684,  0.2383, -0.1021],
         [ 0.3385,  0.0637, -0.2861],
         [-0.1202, -0.2644, -0.0212],
         [-0.3245, -0.0058,  0.1003],
         [-0.0329, -0.2005,  0.0044],
         [ 0.0631, -0.0370, -0.4637],
         [-0.1379, -0.0266, -0.3945],
         [-0.1064,  0.0445, -0.3886]],

        [[-0.0650,  0.3129,  0.1291],
         [-0.2087,  0.1219, -0.2969],
         [-0.0199, -0.1489, -0.0845],
         [-0.1806, -0.2978,  0.1445],
         [ 0.0763,  0.1856, -0.0794],
         [ 0.0233,  0.0700, -0.3206],
         [ 0.1699,  0.0611, -0.0605],
         [-0.0235,  0.0152, -0.2674]]], grad_fn=<TransposeBackward0>)


## Multiclass Text classification

In [11]:
import pandas as pd

In [12]:
#load the data
reviews = pd.read_csv('/content/drive/My Drive/LSTMs/review.csv')
print(reviews.shape)
reviews.head()

(23486, 11)


,Unnamed: 0,Clothing ID,Age,Title,Review Text,Rating,Recommended IND,Positive Feedback Count,Division Name,Department Name,Class Name
0,0,767,33,NaN,Absolutely wonderful - silky and sexy and comf...,4,1,0,Initmates,Intimate,Intimates
1,1,1080,34,NaN,Love this dress! it's sooo pretty. i happene...,5,1,4,General,Dresses,Dresses
2,2,1077,60,Some major design flaws,I had such high hopes for this dress and reall...,3,0,0,General,Dresses,Dresses
3,3,1049,50,My favorite buy!,"I love, love, love this jumpsuit. it's fun, fl...",5,1,0,General Petite,Bottoms,Pants
4,4,847,47,Flattering shirt,This shirt is very flattering to all due to th...,5,1,6,General,Tops,Blouses


In [13]:
reviews['Title'] = reviews['Title'].fillna('')
reviews['Review Text'] = reviews['Review Text'].fillna('')
reviews['review'] = reviews['Title'] + ' ' + reviews['Review Text']

In [14]:
reviews.head()

,Unnamed: 0,Clothing ID,Age,Title,Review Text,Rating,Recommended IND,Positive Feedback Count,Division Name,Department Name,Class Name,review
0,0,767,33,,Absolutely wonderful - silky and sexy and comf...,4,1,0,Initmates,Intimate,Intimates,Absolutely wonderful - silky and sexy and com...
1,1,1080,34,,Love this dress! it's sooo pretty. i happene...,5,1,4,General,Dresses,Dresses,Love this dress! it's sooo pretty. i happen...
2,2,1077,60,Some major design flaws,I had such high hopes for this dress and reall...,3,0,0,General,Dresses,Dresses,Some major design flaws I had such high hopes ...
3,3,1049,50,My favorite buy!,"I love, love, love this jumpsuit. it's fun, fl...",5,1,0,General Petite,Bottoms,Pants,"My favorite buy! I love, love, love this jumps..."
4,4,847,47,Flattering shirt,This shirt is very flattering to all due to th...,5,1,6,General,Tops,Blouses,Flattering shirt This shirt is very flattering...


In [15]:
# keeping only relevent columns and calculating sentence lengths

reviews = reviews[['review', 'Rating']]
reviews.columns = ['review', 'rating']
reviews['review_length'] = reviews['review'].apply(lambda x : len(x.split()))
reviews.head()

,review,rating,review_length
0,Absolutely wonderful - silky and sexy and com...,4,8
1,Love this dress! it's sooo pretty. i happen...,5,62
2,Some major design flaws I had such high hopes ...,3,102
3,"My favorite buy! I love, love, love this jumps...",5,25
4,Flattering shirt This shirt is very flattering...,5,38


In [16]:
#change rating to zero numbering
zero_numbering = {1:0, 2:1, 3:2, 4:3, 5:4}
reviews['rating'] = reviews['rating'].apply(lambda x : zero_numbering[x])

In [17]:
reviews.head()

,review,rating,review_length
0,Absolutely wonderful - silky and sexy and com...,3,8
1,Love this dress! it's sooo pretty. i happen...,4,62
2,Some major design flaws I had such high hopes ...,2,102
3,"My favorite buy! I love, love, love this jumps...",4,25
4,Flattering shirt This shirt is very flattering...,4,38


In [19]:
import numpy as np

In [20]:
#mean sentence length
np.mean(reviews['review_length'])

60.832921740611425

In [32]:
import spacy
import re
import string

In [33]:
#tokenization
tok = spacy.load('en')
def tokenize (text):
    text = re.sub(r"[^\x00-\x7F]+", " ", text)
    regex = re.compile('[' + re.escape(string.punctuation) + '0-9\\r\\t\\n]') # remove punctuation and numbers
    nopunct = regex.sub(" ", text.lower())
    return [token.text for token in tok.tokenizer(nopunct)]

In [34]:
from collections import Counter

In [35]:
#count number of occurences of each word
counts = Counter()
for index, row in reviews.iterrows():
    counts.update(tokenize(row['review']))

In [36]:
#deleting infrequent words
print("num_words before:",len(counts.keys()))
for word in list(counts):
    if counts[word] < 2:
        del counts[word]
print("num_words after:",len(counts.keys()))

num_words before: 14138
num_words after: 8263


In [39]:
#creating vocabulary
vocab2index = {"":0, "UNK":1}
words = ["", "UNK"]
for word in counts:
    vocab2index[word] = len(words)
    words.append(word)

In [40]:
def encode_sentence(text, vocab2index, N=70):
    tokenized = tokenize(text)
    encoded = np.zeros(N, dtype=int)
    enc1 = np.array([vocab2index.get(word, vocab2index["UNK"]) for word in tokenized])
    length = min(N, len(enc1))
    encoded[:length] = enc1[:length]
    return encoded, length

In [41]:
reviews['encoded'] = reviews['review'].apply(lambda x: np.array(encode_sentence(x,vocab2index )))
reviews.head()

,review,rating,review_length,encoded
0,Absolutely wonderful - silky and sexy and com...,3,8,"[[2, 3, 4, 5, 6, 7, 8, 7, 9, 0, 0, 0, 0, 0, 0,..."
1,Love this dress! it's sooo pretty. i happen...,4,62,"[[2, 10, 11, 12, 5, 13, 14, 15, 16, 5, 17, 18,..."
2,Some major design flaws I had such high hopes ...,2,102,"[[54, 55, 56, 57, 17, 58, 59, 60, 61, 62, 11, ..."
3,"My favorite buy! I love, love, love this jumps...",4,25,"[[68, 109, 110, 2, 17, 10, 2, 10, 2, 10, 11, 1..."
4,Flattering shirt This shirt is very flattering...,4,38,"[[122, 123, 11, 123, 52, 92, 122, 19, 124, 125..."


In [42]:
#check how balanced the dataset is
Counter(reviews['rating'])

Counter({0: 842, 1: 1565, 2: 2871, 3: 5077, 4: 13131})

In [43]:
X = list(reviews['encoded'])
y = list(reviews['rating'])
from sklearn.model_selection import train_test_split
X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.2)

In [45]:
from torch.utils.data import Dataset, DataLoader

In [46]:
class ReviewsDataset(Dataset):
    def __init__(self, X, Y):
        self.X = X
        self.y = Y
        
    def __len__(self):
        return len(self.y)
    
    def __getitem__(self, idx):
        return torch.from_numpy(self.X[idx][0].astype(np.int32)), self.y[idx], self.X[idx][1]

In [47]:
train_ds = ReviewsDataset(X_train, y_train)
valid_ds = ReviewsDataset(X_valid, y_valid)

In [55]:
import torch.nn.functional as F
from torch.nn.utils.rnn import pack_padded_sequence, pad_packed_sequence
from sklearn.metrics import mean_squared_error

In [56]:
def train_model(model, epochs=10, lr=0.001):
    parameters = filter(lambda p: p.requires_grad, model.parameters())
    optimizer = torch.optim.Adam(parameters, lr=lr)
    for i in range(epochs):
        model.train()
        sum_loss = 0.0
        total = 0
        for x, y, l in train_dl:
            x = x.long()
            y = y.long()
            y_pred = model(x, l)
            optimizer.zero_grad()
            loss = F.cross_entropy(y_pred, y)
            loss.backward()
            optimizer.step()
            sum_loss += loss.item()*y.shape[0]
            total += y.shape[0]
        val_loss, val_acc, val_rmse = validation_metrics(model, val_dl)
        if i % 5 == 1:
            print("train loss %.3f, val loss %.3f, val accuracy %.3f, and val rmse %.3f" % (sum_loss/total, val_loss, val_acc, val_rmse))

def validation_metrics (model, valid_dl):
    model.eval()
    correct = 0
    total = 0
    sum_loss = 0.0
    sum_rmse = 0.0
    for x, y, l in valid_dl:
        x = x.long()
        y = y.long()
        y_hat = model(x, l)
        loss = F.cross_entropy(y_hat, y)
        pred = torch.max(y_hat, 1)[1]
        correct += (pred == y).float().sum()
        total += y.shape[0]
        sum_loss += loss.item()*y.shape[0]
        sum_rmse += np.sqrt(mean_squared_error(pred, y.unsqueeze(-1)))*y.shape[0]
    return sum_loss/total, correct/total, sum_rmse/total

In [57]:
batch_size = 5000
vocab_size = len(words)
train_dl = DataLoader(train_ds, batch_size=batch_size, shuffle=True)
val_dl = DataLoader(valid_ds, batch_size=batch_size)

In [58]:
class LSTM_fixed_len(torch.nn.Module) :
    def __init__(self, vocab_size, embedding_dim, hidden_dim) :
        super().__init__()
        self.embeddings = nn.Embedding(vocab_size, embedding_dim, padding_idx=0)
        self.lstm = nn.LSTM(embedding_dim, hidden_dim, batch_first=True)
        self.linear = nn.Linear(hidden_dim, 5)
        self.dropout = nn.Dropout(0.2)
        
    def forward(self, x, l):
        x = self.embeddings(x)
        x = self.dropout(x)
        lstm_out, (ht, ct) = self.lstm(x)
        return self.linear(ht[-1])

In [59]:
model_fixed =  LSTM_fixed_len(vocab_size, 50, 50)

In [60]:
train_model(model_fixed, epochs=30, lr=0.01)

train loss 1.261, val loss 1.220, val accuracy 0.559, and val rmse 1.349
train loss 1.193, val loss 1.200, val accuracy 0.560, and val rmse 1.347
train loss 1.146, val loss 1.190, val accuracy 0.556, and val rmse 1.314
train loss 1.029, val loss 1.131, val accuracy 0.566, and val rmse 1.261
train loss 0.897, val loss 1.008, val accuracy 0.606, and val rmse 0.968
train loss 1.024, val loss 0.993, val accuracy 0.575, and val rmse 1.019


In [61]:
train_model(model_fixed, epochs=30, lr=0.01)

train loss 0.889, val loss 0.969, val accuracy 0.607, and val rmse 0.918
train loss 0.814, val loss 0.948, val accuracy 0.620, and val rmse 0.848
train loss 0.770, val loss 0.954, val accuracy 0.618, and val rmse 0.871
train loss 0.719, val loss 0.934, val accuracy 0.628, and val rmse 0.830
train loss 0.667, val loss 0.946, val accuracy 0.634, and val rmse 0.825
train loss 0.622, val loss 0.962, val accuracy 0.638, and val rmse 0.813


In [62]:
train_model(model_fixed, epochs=30, lr=0.01)

train loss 0.634, val loss 1.011, val accuracy 0.623, and val rmse 0.875
train loss 0.564, val loss 0.999, val accuracy 0.633, and val rmse 0.822
train loss 0.518, val loss 1.035, val accuracy 0.631, and val rmse 0.820
train loss 0.484, val loss 1.067, val accuracy 0.633, and val rmse 0.816
train loss 0.452, val loss 1.120, val accuracy 0.632, and val rmse 0.810
train loss 0.416, val loss 1.168, val accuracy 0.633, and val rmse 0.814
